<a href="https://colab.research.google.com/github/satyam26en/TIME_TABLE_KIIT/blob/main/TIME_TABLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
import pandas as pd

# Load the section, professional elective, and core section files from GitHub
section_url = 'https://raw.githubusercontent.com/satyam26en/TIME_TABLE_KIIT/main/SECTION.csv'
elective_url = 'https://raw.githubusercontent.com/satyam26en/TIME_TABLE_KIIT/main/Elective_TIME_TABLE.csv'
core_url = 'https://raw.githubusercontent.com/satyam26en/TIME_TABLE_KIIT/main/CORE_TIME_TABLE_2%20-%20Sheet1.csv'

section_df = pd.read_csv(section_url)
elective_df = pd.read_csv(elective_url)
core_df = pd.read_csv(core_url)

# Normalize the 'Roll No.' column to ensure there are no leading/trailing spaces and consistent data type
section_df['Roll No.'] = section_df['Roll No.'].astype(str).str.strip()

# Define the order of days and a mapping from abbreviated to full day names
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
day_mapping = {
    'MON': 'Monday',
    'TUE': 'Tuesday',
    'WED': 'Wednesday',
    'THU': 'Thursday',
    'FRI': 'Friday',
    'SAT': 'Saturday'
}

# Function to take roll number input and display the timetable
def display_timetable():
    # Take roll number input from the user
    roll_number = input("Enter Roll Number: ").strip()

    # Find the section details for the given roll number
    student_section = section_df[section_df['Roll No.'] == roll_number]

    if student_section.empty:
        print("Roll number not found.")
        return

    # Extract the core section and elective sections
    core_section = student_section['Core Section'].values[0]
    elective_1_section = student_section['Professional Elective 1'].values[0]
    elective_2_section = student_section['Professional Elective 2'].values[0]

    # Retrieve the weekly timetable for Professional Electives 1 and 2
    elective_1_timetable = elective_df[elective_df['Section(DE)'] == elective_1_section]
    elective_2_timetable = elective_df[elective_df['Section(DE)'] == elective_2_section]
    core_timetable = core_df[core_df['Section'] == core_section]

    # Function to format the timetable with subject names
    def format_timetable(timetable_df):
        day_dict = {day: [] for day in day_order}
        room_columns = [col for col in timetable_df.columns if 'ROOM' in col]
        for index, row in timetable_df.iterrows():
            for col in room_columns:
                if row[col] != '---':
                    # Get time column name
                    time_col_index = timetable_df.columns.get_loc(col) + 1
                    time_col = timetable_df.columns[time_col_index]

                    # Get the day and time slot
                    day = row['DAY']
                    day_full = day_mapping.get(day, 'Unknown')
                    if day_full == 'Unknown':
                        continue  # Skip if day is not in the mapping

                    time_slot = time_col

                    # Extract the subject name from the intersection of day and time slot
                    subject = row.get(time_slot, 'N/A')
                    room_number = row[col]

                    if subject.lower() != 'x':  # Only include if it's not 'x'
                        day_dict[day_full].append({
                            'Day': day_full,
                            'Timing': time_slot,
                            'Room Number': room_number,
                            'Subject': subject
                        })
        return day_dict

    # Format the timetables
    elective_1_formatted = format_timetable(elective_1_timetable)
    elective_2_formatted = format_timetable(elective_2_timetable)
    core_formatted = format_timetable(core_timetable)

    # Combine all timetables into day-wise dictionaries
    combined_days = {day: [] for day in day_order}

    for day in day_order:
        combined_days[day].extend(core_formatted[day])
        combined_days[day].extend(elective_1_formatted[day])
        combined_days[day].extend(elective_2_formatted[day])

    # Convert the combined day-wise dictionaries into a single DataFrame
    combined_timetable = pd.DataFrame()
    for day in day_order:
        day_df = pd.DataFrame(combined_days[day])
        combined_timetable = pd.concat([combined_timetable, day_df], ignore_index=True)

    # Display the concatenated and sorted timetable
    print("\nCombined Timetable:")
    print(combined_timetable.to_string(index=False))

    # Save the DataFrame as an Excel file
    combined_timetable.to_excel('timetable.xlsx', index=False)

# Run the function
display_timetable()


Enter Roll Number: 22051615

Combined Timetable:
      Day   Timing Room Number  Subject
   Monday   8 TO 9       B-302       CN
   Monday 9  TO 10       B-302       EE
   Monday 10 TO 11       B-302      DAA
   Monday 11 TO 12    A-DL-102       SE
   Monday  12 TO 1    A-DL-102    CN(L)
   Monday   1 TO 2    A-DL-102    CN(L)
   Monday   4 TO 5       B-402  DOS(DE)
  Tuesday 11 TO 12       B-301       CN
  Tuesday  12 TO 1       B-301      DAA
  Tuesday   1 TO 2       B-301       EE
  Tuesday   4 TO 5       B-203 DMDW(DE)
Wednesday   8 TO 9       B-403       SE
Wednesday 9  TO 10    A-DL-108   DAA(L)
Wednesday 10 TO 11    A-DL-108   DAA(L)
Wednesday 11 TO 12       B-203 DMDW(DE)
 Thursday   3 TO 4       B-402  DOS(DE)
   Friday   8 TO 9       B-305      DAA
   Friday 9  TO 10       B-305       SE
   Friday 10 TO 11       B-305       EE
   Friday   1 TO 2       B-203 DMDW(DE)
   Friday  12 TO 1       B-402  DOS(DE)
 Saturday 10 TO 11       B-302       SE
 Saturday 11 TO 12    A-LH-009 